In [1]:
import cvxpy as cp

# Define the data for the problem
n_buses = 3
n_lines = 2
n_gens = 2
Pd = [40, 30, 20]  # Demand at each bus
X = [0.1, 0.1]      # Line reactances
gen_limits = [(10, 100), (10, 100)]  # (Pmin, Pmax) for each generator
cost_coeffs = [(0.01, 2, 0), (0.015, 1.5, 0)]  # (c, b, a) for each generator

# Define variables
theta = cp.Variable(n_buses)
Pg = cp.Variable(n_gens)

# Objective function
objective = cp.Minimize(sum(cost_coeffs[i][0]*Pg[i]**2 + cost_coeffs[i][1]*Pg[i] + cost_coeffs[i][2] for i in range(n_gens)))

# Power flow expressions
Pf = [(theta[i] - theta[i + 1]) / X[i] for i in range(n_lines)]

# Constraints
constraints = []

# Power balance at each bus
constraints += [Pg[0] - Pd[0] + Pf[0] == 0]  # Bus 1
constraints += [-Pf[0] + Pf[1] - Pd[1] == 0] # Bus 2
constraints += [Pg[1] - Pd[2] - Pf[1] == 0]  # Bus 3

# Generator limits
for i in range(n_gens):
    constraints += [Pg[i] >= gen_limits[i][0], Pg[i] <= gen_limits[i][1]]

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Print results
print("Generator outputs (MW):", Pg.value)
print("Bus angles (radians):", theta.value)
print("Line flows (MW):", [p.value for p in Pf])
print("Optimal Cost ($):", problem.value)


Generator outputs (MW): [44. 46.]
Bus angles (radians): [ 0.35  0.75 -1.85]
Line flows (MW): [-4.000000000000008, 25.999999999999986]
Optimal Cost ($): 208.10000000000002
